In [0]:
from google.colab import drive
from os.path import join

# Create a folder in your drive 'cs236/project' to host in progress code/logs
ROOT = '/content/drive'
PROJ = 'My Drive/cs236/project'  
drive.mount(ROOT)
PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"

In [0]:
GIT_USER = "liamnaka"
GIT_REPO = "space"
GIT_PATH = "https://github.com/{}/{}.git".format(GIT_USER, GIT_REPO)
GIT_BRANCH = "master"

%cd "{PROJECT_PATH}" 
!git clone "{GIT_PATH}"
%cd "{PROJECT_PATH}"/"{GIT_REPO}"
!git checkout "{GIT_BRANCH}" 
!git pull
%cd /content
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

In [0]:
# First, add the official CelebA drive folder from 
# (http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) to your own drive within
# 'cs236/project/data/'
# Unzip CelebA - this should take about a minute

import zipfile
IMG_PATH = 'data/CelebA/Img/img_align_celeba.zip'
with zipfile.ZipFile(join(PROJECT_PATH, IMG_PATH),"r") as zip_ref:
  zip_ref.extractall("data/CelebA")

In [0]:
%cd /content/space
!git submodule sync
!git submodule update --init --remote --recursive
%cd /content

In [0]:
import os
import sys
module_path = os.path.abspath(os.path.join('/content/space/'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('/content/space/HoloGAN'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [0]:
sys.argv[1] = '/content/space/config_ViewHoloGAN.json'

import json
import tensorflow as tf
import numpy as np

from space.model import ViewHoloGAN
from space.HoloGAN.tools.utils import pp, show_all_variables

with open(sys.argv[1], 'r') as fh:
    cfg = json.load(fh)
OUTPUT_DIR = cfg['output_dir']
LOGDIR = os.path.join(OUTPUT_DIR, "log")
os.environ["CUDA_VISIBLE_DEVICES"] = "{0}".format(cfg['gpu'])

flags = tf.app.flags
flags.DEFINE_integer("input_height", 108, "The size of image to use (will be center cropped). [108] or [128] for celebA and lsun, [400] for chairs. Cats and Cars are already cropped")
flags.DEFINE_integer("input_width", None, "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce 64 or 128")
flags.DEFINE_integer("output_width", None, "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "celebA", "The name of dataset [celebA, lsun, chairs, shoes, cars, cats]")
flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_float("train_size", np.inf, "Number of images to train-Useful when only a subset of the dataset is needed to train the model")
flags.DEFINE_boolean("crop", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("rotate_azimuth", False, "Sample images with varying azimuth")
flags.DEFINE_boolean("rotate_elevation", False, "Sample images with varying elevation")
FLAGS = flags.FLAGS





In [0]:
pp.pprint(flags.FLAGS.__flags)
if FLAGS.input_width is None:
    FLAGS.input_width = FLAGS.input_height
if FLAGS.output_width is None:
    FLAGS.output_width = FLAGS.output_height
if not os.path.exists(LOGDIR):
    os.makedirs(LOGDIR)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
print("FLAGs " + str(FLAGS.dataset))
with tf.Session(config=run_config) as sess:
    model = ViewHoloGAN(
        cfg,
        sess,
        input_width=FLAGS.input_width,
        input_height=FLAGS.input_height,
        output_width=FLAGS.output_width,
        output_height=FLAGS.output_height,
        dataset_name=FLAGS.dataset,
        input_fname_pattern=FLAGS.input_fname_pattern,
        crop=FLAGS.crop)

    model.build(cfg['build_func'])

    show_all_variables()

    if FLAGS.train:
        train_func = eval("model." + (cfg['train_func']))
        train_func(FLAGS)
    else:
        if not model.load(LOGDIR)[0]:
            raise Exception("[!] Train a model first, then run test mode")
        sample_func = eval("model." + (cfg['sample_func']))
        sample_func(FLAGS)
